<a href="https://colab.research.google.com/github/mdkamrulhasan/data_mining_kdd/blob/main/notebooks/Exploratory_data_Analysis_Retail_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Today we will learn

1. Data Processing: Aggregation
2. Data visualizations: plotly
3. Table joins, and aggregations from multiple tables


chockolate-promotion.avif

### After completing this notebook, you should be able to answer the following (and/or other similar) quesions (for different datasets):

**General aggregation tasks/questions**

-  What are the different types of stroes available?
- Average size of different store types?
- Average montly sales of store types?
- Association of holidays to store types/sizes?
- Association of temparature to sales?
- Association of fule price to sales?


**Table/data-frame joins:**

- What is the difference among inner/left/right/outer joins?
- What is primary key for your join operations?
- How to do vertical and/or horizontal joins of tables or data-frames.


#### *Note: We will be answering most of the questions above for some specfic data from the Retail domain. Its your time to explore similar questions for other datasets, and you will get a fair list of dataset [here ](https://github.com/mdkamrulhasan/data_mining_kdd/tree/main/data)*

# Importing libraries

In [1]:
# data processing
import pandas as pd
import numpy as np
# visualization
import plotly.express as px
import plotly.graph_objects as go



---



#Loading Data

[Data (Retail) source](https://www.kaggle.com/datasets/manjeetsingh/retaildataset)

In [2]:
# Read the data using pandas
features = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/Features-data-set.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/sales-data-set.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/stores-data-set.csv")

Lets have a first look of the data in each table?

In [3]:
features.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False


In [4]:
sales.head(3)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False


In [5]:
stores.head(3)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392


Question: How many different stores we have?

In [6]:
stores.Store.nunique()

45

#### Question: Can we guess the geolocation of a store?


In [7]:
features['Date'] = pd.to_datetime(features.Date, format="%d/%m/%Y")

Plotting the historical temperature of the location of a store

In [8]:
query_store_id = 1
fig = px.scatter(features[features.Store == query_store_id], x='Date', y='Temperature')
fig.show()

Plotting the historical temperature of the location of a store pair

In [9]:
store_a, store_b = 1,  45 #10, 20
fig = go.Figure([
    go.Scatter(x=features[features.Store == store_a]['Date'],
               y=features[features.Store == store_a]['Temperature']),
    go.Scatter(x = features[features.Store == store_b]['Date'],
               y=features[features.Store == store_b]['Temperature'])
]
               )
fig.update_layout(
    title="Temperature graph", yaxis_title="Temperature", legend_title="store index ")
fig.update_layout(
    legend=dict(
        x=0.05,
        y=0.95,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    )
)
fig.show()

What information can we parse from the stores table ?

In [10]:
stores.head(3)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392


## Aggregation tasks/questions

What are the different types of stroes available?

In [11]:
stores.Type.unique()

array(['A', 'B', 'C'], dtype=object)

Average sizes of store types?

In [12]:
stores.groupby(['Type']).agg({'Size': 'mean'})

,Size
Type,
A,177247.727273
B,101190.705882
C,40541.666667


In [13]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


**Average montly sales of store types?**
- The Stores table doesn't have the sales information
- The Sales table doesn't have the store Type information
- if we can join these tables, we can get both
- Various ways we can join tables using pandas:


> (1) Join (based on indices),

> (2) Merge (more versatile: allows us to specify columns beside index)


Joins
- left
- right
- inner
- outer

In [14]:
# inner join
stores_sales = stores.merge(sales, on=['Store'], how='inner')
stores.shape, sales.shape, stores_sales.shape

((45, 3), (421570, 5), (421570, 7))

In [15]:
stores_sales.head()

,Store,Type,Size,Dept,Date,Weekly_Sales,IsHoliday
0,1,A,151315,1,05/02/2010,24924.50,False
1,1,A,151315,1,12/02/2010,46039.49,True
2,1,A,151315,1,19/02/2010,41595.55,False
3,1,A,151315,1,26/02/2010,19403.54,False
4,1,A,151315,1,05/03/2010,21827.90,False


In [16]:
# left join
stores_sales = stores.merge(sales, on=['Store'], how='left')
stores.shape, sales.shape, stores_sales.shape

((45, 3), (421570, 5), (421570, 7))

In [17]:
# right join
stores_sales = stores.merge(sales, on=['Store'], how='right')
stores.shape, sales.shape, stores_sales.shape

((45, 3), (421570, 5), (421570, 7))

### Lets create a "Synthetic" case?

In [18]:
my_synthetic_stores = stores
my_synthetic_stores['Store'] = my_synthetic_stores['Store'] + 100
my_synthetic_stores.Store.unique()

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145])

In [19]:
my_synthetic_stores.head()

,Store,Type,Size
0,101,A,151315
1,102,A,202307
2,103,B,37392
3,104,A,205863
4,105,B,34875


In [20]:
# left join
my_synthetic_stores_sales = my_synthetic_stores.merge(sales, on=['Store'], how='left')
my_synthetic_stores.shape#, sales.shape, my_synthetic_stores.shape

(45, 3)

In [21]:
my_synthetic_stores_sales.head(2)

,Store,Type,Size,Dept,Date,Weekly_Sales,IsHoliday
0,101,A,151315,NaN,NaN,NaN,NaN
1,102,A,202307,NaN,NaN,NaN,NaN


You can play with the above data from other joins: inner/right/outer etc.

Average sales of each store types

In [22]:
stores_sales.head()

,Store,Type,Size,Dept,Date,Weekly_Sales,IsHoliday
0,1,A,151315,1,05/02/2010,24924.50,False
1,1,A,151315,1,12/02/2010,46039.49,True
2,1,A,151315,1,19/02/2010,41595.55,False
3,1,A,151315,1,26/02/2010,19403.54,False
4,1,A,151315,1,05/03/2010,21827.90,False


In [23]:
stores_sales.groupby(['Type']).agg({'Weekly_Sales': 'mean'})

,Weekly_Sales
Type,
A,20099.568043
B,12237.075977
C,9519.532538


General aggregation tasks/questions
-  What are the different types of stroes available?
- Average size of different store types?
- Average montly sales of store types?
- Association of holidays to store types/sizes?
- Association of temparature to sales?
- Association of fule price to sales?

# Concatention of dataframes

In [24]:
stores.shape

(45, 3)

In [25]:
s1s2 = pd.concat([stores, stores], axis=0)
s1s2.shape

(90, 3)

In [28]:
s1s2_v = pd.concat([stores, stores], axis=1)
s1s2_v.shape

(45, 6)

In [29]:
s1s2_v.head()

,Store,Type,Size,Store,Type,Size
0,101,A,151315,101,A,151315
1,102,A,202307,102,A,202307
2,103,B,37392,103,B,37392
3,104,A,205863,104,A,205863
4,105,B,34875,105,B,34875
